In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature

In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";"
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
### split data into training, testing and validation

train,test = train_test_split(data,test_size=0.25,random_state=42)

In [5]:
train_x = train.drop(['quality'],axis=1).values
train_y = train['quality'].values.ravel()

## testing dataset
test_x = test.drop(['quality'],axis=1).values
test_y = test['quality'].values.ravel()

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

signature = infer_signature(train_x,train_y)

In [11]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## model architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean,variance=var),
        keras.layers.Dense(64,activation='relu'),
        keras.layers.Dense(1)
    ])


    ## compile the model 
    model.compile(optimizer=keras.optimizers.SGD(
                  learning_rate = params['lr'], momentum = params['momentum']
                  ),
                  loss='mean_squared_error',
                  metrics =[ keras.metrics.RootMeanSquaredError()]
                  )
    
    ## Train ANN model with lr and momentum with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=32)
        
        ##evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=32)

        eval_rmse = eval_result[1]

        ## log the parameteres and results
        # Log each parameter
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)
        mlflow.log_metric('rmse',eval_rmse)

        ### log model 
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {'loss':eval_rmse,"status":STATUS_OK,"model":model}

In [12]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x= train_x,
        train_y= train_y,
        test_x= test_x,
        valid_x=valid_x,
        valid_y=valid_y,
        test_y=test_y
    )
    return result

In [13]:
space = {
    'lr' :hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    'momentum':hp.uniform("momentum",0.0,1.0)
}

In [16]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # conduct the hyperparameter search using hpyeropt
    trails = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo= tpe.suggest,
        max_evals=4,
        trials=trails
    )
    

    # fetch the details of best run
    best_run = sorted(trails.results,key=lambda x:x['loss'])[0]

    # log the best parameters
    # Log each parameter
    for param_name, param_value in best.items():
        mlflow.log_param(param_name, param_value)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run['model'],"model",signature=signature)

    print(f"Best parameters: {best}")
    print(f"Best Eval Rmse: {best_run['loss']}")

2025/01/02 19:10:02 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 39s 438ms/step - loss: 35.6628 - root_mean_squared_error: 5.9718
35/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.9313 - root_mean_squared_error: 5.5598   
69/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.5208 - root_mean_squared_error: 5.2316
90/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.2682 - root_mean_squared_error: 4.9967
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 24.9673 - root_mean_squared_error: 4.9643 - val_loss: 3.8191 - val_root_mean_squared_error: 1.9542

Epoch 2/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 3.4789 - root_mean_squared_error: 1.8652
32/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.7126 - root_mean_squared_error: 2.1605 
64/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5760 - root_mean_squared_error: 2.3488
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7763 - root_mean_squared_error: 2.3938 - val_loss: 2.3833 - val_ro

In [17]:
## inferencing
from mlflow.models import validate_serving_input

model_uri = 'runs:/8159df3f0030465b9011aebc9b9d80db/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\moiz.DESKTOP-EOSKFCV\anaconda3\envs\mlops\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[4.176057 ],
       [6.7754583],
       [5.7462425],
       ...,
       [7.449068 ],
       [6.9104137],
       [4.769905 ]], dtype=float32)